## Deploy your pre-trained keras model to AWS
adapted from https://aws.amazon.com/blogs/machine-learning/deploy-trained-keras-or-tensorflow-models-using-amazon-sagemaker/

In [1]:
import boto3, re
from sagemaker import get_execution_role

role = get_execution_role()

## Upload your models onto the SageMaker instance
- this can be done manually by simply clicking 'upload' button on jupyter 'home' screen

In [2]:
## LOAD MODEL
model_location = 'model.h5'

# load a saved model
#import tensorflow
from keras.models import load_model
 
# load model
model = load_model(model_location)

Using TensorFlow backend.













Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




In [5]:
def convert_h5_to_aws(loaded_model):
    """
    given a pre-trained keras model, this function converts it to a TF protobuf format
    and saves it in the file structure which aws expects
    """
    
    from tensorflow.python.saved_model import builder
    from tensorflow.python.saved_model.signature_def_utils import predict_signature_def
    from tensorflow.python.saved_model import tag_constants
    
    # This is the file structure which AWS expects. Cannot be changed. 
    model_version = '1'
    export_dir = 'export/Servo/' + model_version
    
    # Build the Protocol Buffer SavedModel at 'export_dir'
    builder = builder.SavedModelBuilder(export_dir)
    
    # Create prediction signature to be used by TensorFlow Serving Predict API
    signature = predict_signature_def(
        inputs={"inputs": loaded_model.input}, outputs={"score": loaded_model.output})
    
    from keras import backend as K
    with K.get_session() as sess:
        # Save the meta graph and variables
        builder.add_meta_graph_and_variables(
            sess=sess, tags=[tag_constants.SERVING], signature_def_map={"serving_default": signature})
        builder.save()
    
    #create a tarball/tar file and zip it
    import tarfile
    with tarfile.open('model.tar.gz', mode='w:gz') as archive:
        archive.add('export', recursive=True)
        
convert_h5_to_aws(model)

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: export/Servo/1/saved_model.pb


## Move the tarball (tar.gz)
- to a folder called 'model' which is 
- in an S3 bucket

In [7]:
import sagemaker

sagemaker_session = sagemaker.Session()
inputs = sagemaker_session.upload_data(path='model.tar.gz', key_prefix='model')

This is the name of the bucket which SageMaker made in S3

In [8]:
# where did it upload to?
print("Bucket name is:")
sagemaker_session.default_bucket()

Bucket name is:


'sagemaker-us-east-1-442707538511'

## Deployment Time!
First, create an empty train.py file (TensorFlowModel expects this at its 'entry point'

In [9]:
!touch train.py

In [10]:
%%timeit
from sagemaker.tensorflow.model import TensorFlowModel
sagemaker_model = TensorFlowModel(model_data = 's3://' + sagemaker_session.default_bucket() + '/model/model.tar.gz',
                                  role = role,
                                  framework_version = '1.12',
                                  entry_point = 'train.py')

predictor = sagemaker_model.deploy(initial_instance_count=1,
                                   instance_type='ml.m4.xlarge')

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


-----------------!

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


----------

KeyboardInterrupt: 

In [53]:
# What is our endpoing called?
predictor.endpoint

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


'sagemaker-tensorflow-serving-2021-02-03-07-04-15-657'

In [54]:
# Create a predictor which uses this new endpoint
import sagemaker
from sagemaker.tensorflow.model import TensorFlowModel
predictor=sagemaker.tensorflow.model.TensorFlowPredictor(predictor.endpoint, sagemaker_session)

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [58]:
import numpy as np

# The sample model expects an input of a multi-dimennsional array of 4 inputs
data = np.asarray([[5. , 3.5, 1.3, 0.3]])
predictor.predict(data)

# # Locally, with the input below we get the following predictions:
# expected predictions: 
#           [[9.9930930e-01, 6.9067377e-04, 1.5728773e-16]]

{'predictions': [[0.999309, 0.000690674, 1.57288e-16]]}

## Cleanup:
https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-cleanup.html

- Stop Notebook
- delete endpoints
- delete models
- delete S3 bucket
- delete cloudwatch groups?